# Lab | Simple LLM App with LCEL

In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

### LangSmith

Many of the applications you build with LangChain will contain multiple steps with multiple invocations of LLM calls.
As these applications get more and more complex, it becomes crucial to be able to inspect what exactly is going on inside your chain or agent.
The best way to do this is with [LangSmith](https://smith.langchain.com).

After you sign up at the link above, make sure to set your environment variables to start logging traces:

```shell
export LANGCHAIN_TRACING_V2="true"
export LANGCHAIN_API_KEY="..."
```

Or, if in a notebook, you can set them with:

```python
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()
```

In [14]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_community.chat_models import ChatOllama

# Use Ollama with a local model (requires Ollama to be installed)
model = ChatOllama(model="llama2")  # or another model like "mistral" or "gemma"

C:\Users\rfria\AppData\Local\Temp\ipykernel_21224\2481728250.py:5: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  model = ChatOllama(model="llama2")  # or another model like "mistral" or "gemma"


Let's first use the model directly. `ChatModel`s are instances of LangChain "Runnables", which means they expose a standard interface for interacting with them. To just simply call the model, we can pass in a list of messages to the `.invoke` method.

In [17]:
messages = [
    SystemMessage(content="Translate the following from English into Italian"), 
    HumanMessage(content="Hello, how are you today?"),
]

model.invoke(messages)

AIMessage(content='\nCiao! Come stai oggi? (CHOW koh-LAY COO-ahyay OH-dee)\n\nNote:\n\n* "Ciao" is a common greeting in Italian, equivalent to "hello" in English.\n* "Come stai?" is a question asking how someone is doing, and can be translated as "How are you?" or "How are things going?"\n* "Oggi" means "today".', additional_kwargs={}, response_metadata={'model': 'llama2', 'created_at': '2025-03-31T15:02:04.6915923Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 35813161200, 'load_duration': 32167058100, 'prompt_eval_count': 35, 'prompt_eval_duration': 641163900, 'eval_count': 100, 'eval_duration': 2996780300}, id='run-816e8109-eb23-44f2-99dc-6218482a1be5-0')

In [18]:
messages = [
    SystemMessage(content="Translate the following from English into Italian"), 
    HumanMessage(content="Hello, how are you today?"),
]

model.invoke(messages)

AIMessage(content='\nCiao! Come stai oggi? (CHOW koh, KOH-may STAH-ee)\n\nLiteral translation:\n\nHello! How are you today?\n\nNote: In Italian, "ciao" is a common greeting that can be used at any time of day, while "come stai?" is a question asking about someone\'s well-being.', additional_kwargs={}, response_metadata={'model': 'llama2', 'created_at': '2025-03-31T15:02:19.1344945Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 2909668400, 'load_duration': 12790900, 'prompt_eval_count': 35, 'prompt_eval_duration': 11641800, 'eval_count': 87, 'eval_duration': 2884014400}, id='run-fe22ea9d-7178-4286-99e6-6d7b4c99dc73-0')

In [19]:
messages = [
    SystemMessage(content="Translate the following from English into Italian"), 
    HumanMessage(content="Hello, how are you today?"),
]

model.invoke(messages)

AIMessage(content='\nCiao! Come stai oggi? (CHOW-oh! KOH-may STY-ee OH-dee)\n\nExplanation:\n\n* "Ciao" is a common Italian greeting, equivalent to "hello" in English.\n* "Come stai?" is a question asking how someone is doing, which can be translated as "How are you today?" in English.', additional_kwargs={}, response_metadata={'model': 'llama2', 'created_at': '2025-03-31T15:02:27.8000871Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 3164524200, 'load_duration': 34928300, 'prompt_eval_count': 35, 'prompt_eval_duration': 19144000, 'eval_count': 89, 'eval_duration': 3107797400}, id='run-89a0b51e-9fc0-4585-8982-49f0ac19733c-0')

In [20]:
messages = [
    SystemMessage(content="Translate the following from English into Italian"), 
    HumanMessage(content="Hello, how are you today?"),
]

model.invoke(messages)

AIMessage(content='\nCiao! Come stai oggi?', additional_kwargs={}, response_metadata={'model': 'llama2', 'created_at': '2025-03-31T15:02:43.9134478Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 583274700, 'load_duration': 11068200, 'prompt_eval_count': 35, 'prompt_eval_duration': 11834700, 'eval_count': 11, 'eval_duration': 559309100}, id='run-991765e3-f53a-492e-80ec-d0f33137909a-0')

If we've enable LangSmith, we can see that this run is logged to LangSmith, and can see the [LangSmith trace](https://smith.langchain.com/public/88baa0b2-7c1a-4d09-ba30-a47985dde2ea/r)

## OutputParsers

Notice that the response from the model is an `AIMessage`. This contains a string response along with other metadata about the response. Oftentimes we may just want to work with the string response. We can parse out just this response by using a simple output parser.

We first import the simple output parser.

In [21]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

One way to use it is to use it by itself. For example, we could save the result of the language model call and then pass it to the parser.

In [22]:
result = model.invoke(messages)

In [23]:
parser.invoke(result)

'\nCiao! Come stai oggi?'

More commonly, we can "chain" the model with this output parser. This means this output parser will get called everytime in this chain. This chain takes on the input type of the language model (string or list of message) and returns the output type of the output parser (string).

We can easily create the chain using the `|` operator. The `|` operator is used in LangChain to combine two elements together.

In [34]:
chain = None

In [30]:
chain.invoke(messages)

AttributeError: 'AIMessage' object has no attribute 'invoke'

If we now look at LangSmith, we can see that the chain has two steps: first the language model is called, then the result of that is passed to the output parser. We can see the [LangSmith trace]( https://smith.langchain.com/public/f1bdf656-2739-42f7-ac7f-0f1dd712322f/r)

## Prompt Templates

Right now we are passing a list of messages directly into the language model. Where does this list of messages come from? Usually, it is constructed from a combination of user input and application logic. This application logic usually takes the raw user input and transforms it into a list of messages ready to pass to the language model. Common transformations include adding a system message or formatting a template with the user input.

PromptTemplates are a concept in LangChain designed to assist with this transformation. They take in raw user input and return data (a prompt) that is ready to pass into a language model. 

Let's create a PromptTemplate here. It will take in two user variables:

- `language`: The language to translate text into
- `text`: The text to translate

In [35]:
from langchain_core.prompts import ChatPromptTemplate

First, let's create a string that we will format to be the system message:

In [36]:
system_template = "Translate the following into {language}:"

Next, we can create the PromptTemplate. This will be a combination of the `system_template` as well as a simpler template for where the put the text

In [37]:
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

The input to this prompt template is a dictionary. We can play around with this prompt template by itself to see what it does by itself

In [39]:
result = prompt_template.invoke({"language": "italian", "text": "how are you doing today?"})

result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into italian:', additional_kwargs={}, response_metadata={}), HumanMessage(content='how are you doing today?', additional_kwargs={}, response_metadata={})])

We can see that it returns a `ChatPromptValue` that consists of two messages. If we want to access the messages directly we do:

In [40]:
result.to_messages()

[SystemMessage(content='Translate the following into italian:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='how are you doing today?', additional_kwargs={}, response_metadata={})]

## Chaining together components with LCEL

We can now combine this with the model and the output parser from above using the pipe (`|`) operator:

In [41]:
chain = None

In [42]:
chain.invoke({"language": "italian", "text": None})

AttributeError: 'NoneType' object has no attribute 'invoke'

This is a simple example of using [LangChain Expression Language (LCEL)](/docs/concepts/#langchain-expression-language-lcel) to chain together LangChain modules. There are several benefits to this approach, including optimized streaming and tracing support.

If we take a look at the LangSmith trace, we can see all three components show up in the [LangSmith trace](https://smith.langchain.com/public/bc49bec0-6b13-4726-967f-dbd3448b786d/r).

## Serving with LangServe

Now that we've built an application, we need to serve it. That's where LangServe comes in.
LangServe helps developers deploy LangChain chains as a REST API. You do not need to use LangServe to use LangChain, but in this guide we'll show how you can deploy your app with LangServe.

While the first part of this guide was intended to be run in a Jupyter Notebook or script, we will now move out of that. We will be creating a Python file and then interacting with it from the command line.

Install with:
```bash
pip install "langserve[all]"
```

### Server

To create a server for our application we'll make a `serve.py` file. This will contain our logic for serving our application. It consists of three things:
1. The definition of our chain that we just built above
2. Our FastAPI app
3. A definition of a route from which to serve the chain, which is done with `langserve.add_routes`


```python
#!/usr/bin/env python
from typing import List

from fastapi import FastAPI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langserve import add_routes

# 1. Create prompt template
system_template = "Translate the following into {language}:"
prompt_template = ChatPromptTemplate.from_messages([
    ('system', system_template),
    ('user', '{text}')
])

# 2. Create model
model = ChatOpenAI()

# 3. Create parser
parser = StrOutputParser()

# 4. Create chain
chain = prompt_template | model | parser


# 4. App definition
app = FastAPI(
  title="LangChain Server",
  version="1.0",
  description="A simple API server using LangChain's Runnable interfaces",
)

# 5. Adding chain route

add_routes(
    app,
    chain,
    path="/chain",
)

if __name__ == "__main__":
    import uvicorn

    uvicorn.run(app, host="localhost", port=8000)
```

And that's it! If we execute this file:
```bash
python serve.py
```
we should see our chain being served at [http://localhost:8000](http://localhost:8000).

### Playground

Every LangServe service comes with a simple [built-in UI](https://github.com/langchain-ai/langserve/blob/main/README.md#playground) for configuring and invoking the application with streaming output and visibility into intermediate steps.
Head to [http://localhost:8000/chain/playground/](http://localhost:8000/chain/playground/) to try it out! Pass in the same inputs as before - `{"language": "italian", "text": "hi"}` - and it should respond same as before.

### Client

Now let's set up a client for programmatically interacting with our service. We can easily do this with the `[langserve.RemoteRunnable](/docs/langserve/#client)`.
Using this, we can interact with the served chain as if it were running client-side.

In [5]:
from langserve import RemoteRunnable

remote_chain = RemoteRunnable("http://localhost:8000/chain/")
remote_chain.invoke({"language": "japanese", "text": "Hello, how are you today?"})  # Replace None with an actual string

'\nこんにちは、今日はいかがですか？(Konnichiwa, kyou wa ikaga desu ka?)\n\nHere\'s a breakdown of the sentence and how it would be translated in Japanese:\n\n* "Hello" is こんにちは (konnichiwa) in Japanese.\n* "How are you today?" is 今日はいかがですか (kyou wa ikaga desu ka) in Japanese. The word order is different in Japanese, and the phrase "how are you" is not directly translated as a single word, but rather as "いかがですか" which can be translated to "How are you?" or "How are things going?" depending on the context.'

To learn more about the many other features of LangServe [head here](/docs/langserve).

## Conclusion

That's it! In this tutorial you've learned how to create your first simple LLM application. You've learned how to work with language models, how to parse their outputs, how to create a prompt template, chaining them with LCEL, how to get great observability into chains you create with LangSmith, and how to deploy them with LangServe.

This just scratches the surface of what you will want to learn to become a proficient AI Engineer. Luckily - we've got a lot of other resources!

For further reading on the core concepts of LangChain, we've got detailed [Conceptual Guides](/docs/concepts).

If you have more specific questions on these concepts, check out the following sections of the how-to guides:

- [LangChain Expression Language (LCEL)](/docs/how_to/#langchain-expression-language-lcel)
- [Prompt templates](/docs/how_to/#prompt-templates)
- [Chat models](/docs/how_to/#chat-models)
- [Output parsers](/docs/how_to/#output-parsers)
- [LangServe](/docs/langserve/)

And the LangSmith docs:

- [LangSmith](https://docs.smith.langchain.com)